In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
df = pd.read_csv('maSVk18.csv')
Code_list = df['0'].tolist()
last_semester_listCode = []
mark_last_semester = []
credit_list = []
graduate_type_list = []
for Code in Code_list:
    url = 'http://thongtindaotao.sgu.edu.vn/Default.aspx?page=xemdiemthi&id='
    URL = url + str(Code)
    response = requests.get(URL)
    page_source = BeautifulSoup(response.content)
    mark = page_source.find_all('td',align = 'left',colspan = '14')
    last_semester = mark[len(mark)-8]
    mark_semester = mark[len(mark)-4]
    credit = mark[len(mark)-2]
    last_semester = last_semester.get_text()
    mark_semester = mark_semester.get_text()
    credit = credit.get_text()
    if last_semester == 'Học kỳ 1 - Năm học 2021-2022':
        last_semester_listCode.append(Code)
        mark_last_semester.append(mark_semester[-4:])
        credit = credit[-3:]
        if ':' in credit:
            credit_list.append(credit[-2:])
        else:
            credit_list.append(credit[-3:])
    subjects = page_source.find_all('td',align = 'left',width = '250')
    MonThayThe = ['Seminar chuyên đề','	Các công nghệ lập trình hiện đại','Khai phá dữ liệu']
    flag = False
    for subject in subjects:
        subject_name = subject.get_text().strip()
        if subject_name in MonThayThe:
            graduate_type_list.append('Mon Thay The')
            flag = True
            break
        elif subject_name == 'Khóa luận tốt nghiệp (DCT)':
            graduate_type_list.append('Khoa luan')
            flag = True
            break
    if flag == False:
        graduate_type_list.append('Chua xac dinh')
dict = {'ma SV':last_semester_listCode,'diem':mark_last_semester,'tin tich luy':credit_list,'loai tot nghiep':graduate_type_list}
df = pd.DataFrame(dict)
df.to_csv('diemk18.csv',index = False)
